In [1]:
!pip install gdown
!gdown --id 1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA
!unzip mangodetection.zip
!mv /content/content/mangodetection /content/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA
To: /content/mangodetection.zip
100% 767M/767M [00:08<00:00, 92.4MB/s]
Archive:  mangodetection.zip
   creating: content/mangodetection/
   creating: content/mangodetection/test/
   creating: content/mangodetection/test/healthy/
  inflating: content/mangodetection/test/healthy/0001_0137.JPG  
  inflating: content/mangodetection/test/healthy/0001_0072.JPG  
  inflating: content/mangodetection/test/healthy/0001_0152.JPG  
  inflating: content/mangodetection/test/healthy/0001_0092.JPG  
  inflating: content/mangodetection/test/healthy/0001_0004.JPG  
  inflating: content/mangodetection/test/heal

In [2]:
import tensorflow as tf
import IPython.display as display
import PIL
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [3]:
data_dir = pathlib.Path('/content/mangodetection/train')
test_dir = pathlib.Path('/content/mangodetection/test')
train_count = len(list(data_dir.glob('*/*.JPG')))
test_count = len(list(test_dir.glob('*/*.JPG')))

batch_size = 32
img_height = 180
img_width = 180

#Generate More Data by Generator
image_gen_train = ImageDataGenerator(rescale = 1./255,validation_split=0.2,rotation_range = 45,width_shift_range=.15,height_shift_range =.15,horizontal_flip=True,zoom_range=0.5)

#Shuffle All Data 
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'training',  directory= data_dir,shuffle=True,target_size=(img_height, img_width))  

#Divide Data for Validation
validation_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'validation',  directory= data_dir,shuffle=True,target_size=(img_height, img_width)) 

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

Found 330 images belonging to 2 classes.
Found 82 images belonging to 2 classes.


In [4]:
num_classes = 2
model = Sequential([
  Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width ,3)),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Dropout(0.2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(num_classes)
])
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 64)       0

In [5]:
model_mango = load_model('/content/mangodetection/model.h5')

image_gen_test = ImageDataGenerator(rescale = 1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=32,directory= test_dir,shuffle=True,target_size=(img_height, img_width))  

start_time = time.time()
predictions = model_mango.predict(test_data_gen)
for i in range(len(predictions)):
    if(predictions[i][0]<0):
      print("Healthy     "+str(abs(predictions[i][0]))+"%")
    else:  
      print("Diseased    "+str(100-abs(predictions[i][0]))+"%")
print("Total Time: ", time.time()-start_time)
print("Average Time Per Image: ", (time.time()-start_time)/len(predictions))

Found 23 images belonging to 2 classes.
1/1 [==============================] - 10s 10s/step
Healthy     6.717607%
Healthy     7.960397%
Diseased    88.71868228912354%
Diseased    87.19365310668945%
Diseased    93.6432147026062%
Healthy     7.11954%
Diseased    87.38347721099854%
Diseased    85.0113296508789%
Diseased    90.07216739654541%
Healthy     7.732728%
Diseased    86.62044334411621%
Diseased    91.33833503723145%
Healthy     6.482184%
Diseased    86.64076805114746%
Healthy     6.995202%
Diseased    89.65610408782959%
Healthy     6.9669805%
Diseased    87.49136924743652%
Diseased    88.4575252532959%
Healthy     7.206076%
Healthy     6.0367737%
Diseased    92.68936681747437%
Diseased    93.44462585449219%
Total Time:  20.30818223953247
Average Time Per Image:  0.8829687678295634


[https://diseaseidentification.anvil.app](https://diseaseidentification.anvil.app)

In [ ]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("SNO5N6UNOGXMICYSINU4DIEJ-AFBDS2UOAU7ZPMEU")
import anvil.media
import numpy as np

from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import save_img
from tensorflow.keras.utils import img_to_array

import cv2
%cd /content
!mkdir tst
%cd tst
!mkdir test
%cd /content
@anvil.server.callable
def imgload(file):
  model_mango = load_model('/content/mangodetection/model.h5')
  with anvil.media.TempFile(file) as filename:
    img = load_img(filename)
    img_array = img_to_array(img)
    save_img('/content/tst/test/temp.png',img_array)
    print("Processed Incoming Image")
    
  image_gen_test = ImageDataGenerator(rescale = 1./255)
  test_data_gen = image_gen_test.flow_from_directory(batch_size=1,directory= "/content/tst",shuffle=True,target_size=(img_height, img_width)) 
  predictions = model_mango.predict(test_data_gen) 
  print(test_data_gen,predictions)
  li=''
  if(predictions[0][0]<0):
      li='Healthy'
  else:  
    li="Diseased"
  return(li)
anvil.server.wait_forever()  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Reconnecting Anvil Uplink...
Connecting to https://diseaseidentification.anvil.app
Reconnection failed. Waiting 10 seconds, then retrying.
Reconnecting Anvil Uplink...
Connecting to https://diseaseidentification.anvil.app
Reconnection failed. Waiting 10 seconds, then retrying.


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER
/content
mkdir: cannot create directory ‘tst’: File exists
/content/tst
mkdir: cannot create directory ‘test’: File exists
/content
Reconnecting Anvil Uplink...
Reconnecting Anvil Uplink...
